# Study TensorFlow 

[公式のGet Started](https://www.tensorflow.org/get_started/get_started) を使って勉強。

##  1次方程式の推定

### モデルの作成 & 初期値の設定

In [57]:
import tensorflow as tf

W = tf.Variable([.3], tf.float32)
b = tf.Variable([-.3], tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W * x + b

init = tf.global_variables_initializer()  # ここで変数を全て初期化

$W=0.3$

$b=0.3$

$linermodel = W * x + b$

### モデルに複数の値を代入し、結果を確認

In [58]:
sess = tf.Session()   # 実際の計算のセッションを開始
sess.run(init)   # 変数の初期化をセッションに反映
print(sess.run(linear_model, {x:[1,2,3,4]}))  # 初期値とxに代入する複数の値に従って、モデルを計算

[ 0.          0.30000001  0.60000002  0.90000004]


### 損失関数の定義と損失関数への複数の値の代入

In [59]:
y = tf.placeholder(tf.float32)
# 損失関数の定義
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)
print(sess.run(loss, {x:[1,2,3,4], y:[0,-1,-2,-3]}))

23.66


### 手動によるパラメータの修正

In [60]:
fixW = tf.assign(W, [-1.])
fixb = tf.assign(b, [1.])
sess.run([fixW, fixb])  # パラメータの修正
print(sess.run(loss, {x:[1,2,3,4], y:[0,-1,-2,-3]}))   # 損失関数, 代入値のlist, 目標値のllist

0.0


### 勾配降下法による最適化

In [61]:
# パラメータを元の値に修正
fixW = tf.assign(W, [3.])
fixb = tf.assign(b, [3.])
sess.run([fixW, fixb])

optimizer = tf.train.GradientDescentOptimizer(0.01) #勾配降下法による最適化。0.01は学習率(どれだけパラメータを更新するか)。
train = optimizer.minimize(loss)
sess.run(init) # reset values to incorrect defaults.
for i in range(1000):
  sess.run(train, {x:[1,2,3,4], y:[0,-1,-2,-3]})

print(sess.run([W, b]))

[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]


$W=-1, b=1$ という"解"にとても近似した結果が得られる。

* [TensorFlowのOptimizer](http://qiita.com/TomokIshii/items/f355d8e87d23ee8e0c7a)

|   Optimizer name  |  Description   |
|:-------------------|:---------------|
| GradientDescentOptimizer | 勾配降下法によるオプティマイザー
| AdagradOptimizer |  AdaGrad法によるオプティマイザー |
| MomentumOptimizer | モメンタム法によるオプティマイザー  |
| AdamOptimizer | Adam法 （これも有名ですね．）|
| FtrlOptimizer |（初めて耳にしました）"Follow the Regularized Leader" アルゴリズム  |
| RMSPropOptimizer | （初めて耳にしました）学習率の調整を自動化したアルゴリズム  |

### まとめ

In [62]:
import numpy as np
import tensorflow as tf

# Model parameters
W = tf.Variable([.3], tf.float32)
b = tf.Variable([-.3], tf.float32)
# Model input and output
x = tf.placeholder(tf.float32)
linear_model = W * x + b
y = tf.placeholder(tf.float32)
# loss
loss = tf.reduce_sum(tf.square(linear_model - y)) # sum of the squares
# optimizer
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)
# training data
x_train = [1,2,3,4]
y_train = [0,-1,-2,-3]
# training loop
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init) # reset values to wrong
for i in range(1000):
  sess.run(train, {x:x_train, y:y_train})

# evaluate training accuracy
curr_W, curr_b, curr_loss  = sess.run([W, b, loss], {x:x_train, y:y_train})
print("W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))

W: [-0.9999969] b: [ 0.99999082] loss: 5.69997e-11


## ハイレベルのライブラリを用いた線形回帰の実装

In [63]:
#import tensorflow as tf
# NumPy is often used to load, manipulate and preprocess data.
#import numpy as np

# Declare list of features. We only have one real-valued feature. There are many
# other types of columns that are more complicated and useful.
features = [tf.contrib.layers.real_valued_column("x", dimension=1)]

# An estimator is the front end to invoke training (fitting) and evaluation
# (inference). There are many predefined types like linear regression,
# logistic regression, linear classification, logistic classification, and
# many neural network classifiers and regressors. The following code
# provides an estimator that does linear regression.
estimator = tf.contrib.learn.LinearRegressor(feature_columns=features)

# TensorFlow provides many helper methods to read and set up data sets.
# Here we use `numpy_input_fn`. We have to tell the function how many batches
# of data (num_epochs) we want and how big each batch should be.
x = np.array([1., 2., 3., 4.])
y = np.array([0., -1., -2., -3.])
input_fn = tf.contrib.learn.io.numpy_input_fn({"x":x}, y, batch_size=4,
                                              num_epochs=1000)

# We can invoke 1000 training steps by invoking the `fit` method and passing the
# training data set.
estimator.fit(input_fn=input_fn, steps=1000)

# Here we evaluate how well our model did. In a real example, we would want
# to use a separate validation and testing data set to avoid overfitting.
estimator.evaluate(input_fn=input_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1101e7320>, '_master': '', '_num_ps_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000}
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/18/d3ht3py

{'global_step': 1000, 'loss': 1.8092764e-07}

### ローレベルAPIを用いてハイレベルAPIのモデルを生成する方法

In [64]:
import numpy as np
import tensorflow as tf
# Declare list of features, we only have one real-valued feature
def model(features, labels, mode):
  # Build a linear model and predict values
  W = tf.get_variable("W", [1], dtype=tf.float64)
  b = tf.get_variable("b", [1], dtype=tf.float64)
  y = W*features['x'] + b
  # Loss sub-graph
  loss = tf.reduce_sum(tf.square(y - labels))
  # Training sub-graph
  global_step = tf.train.get_global_step()
  optimizer = tf.train.GradientDescentOptimizer(0.01)
  train = tf.group(optimizer.minimize(loss),
                   tf.assign_add(global_step, 1))
  # ModelFnOps connects subgraphs we built to the
  # appropriate functionality.
  return tf.contrib.learn.ModelFnOps(
      mode=mode, predictions=y,
      loss=loss,
      train_op=train)

estimator = tf.contrib.learn.Estimator(model_fn=model)
# define our data set
x = np.array([1., 2., 3., 4.])
y = np.array([0., -1., -2., -3.])
input_fn = tf.contrib.learn.io.numpy_input_fn({"x": x}, y, 4, num_epochs=1000)

# train
estimator.fit(input_fn=input_fn, steps=1000)
# evaluate our model
print(estimator.evaluate(input_fn=input_fn, steps=10))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x10cde4da0>, '_master': '', '_num_ps_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/18/d3ht3pyn34l44kfnkbmf_7180000gn/T/tmps_qvath2/model.ckpt.
INFO:tensorflow:loss = 1.02906037593, step = 1
INFO:tensorflow:global_step/sec: 1141.5
INFO:tensorflow:loss = 0.364970988348, step = 101
INFO:tensorflow:global_step/sec: 879.608
INFO:tensorflow:loss = 0.0233165410683, step = 201
INFO:tensorflow:global_step/sec: 818.318
INFO:tens